In [1]:
def editDistance(str1, str2, m, n):
    if m == 0:
        return n
    if n == 0:
        return m


    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)

    return 1 + min(editDistance(str1, str2, m, n-1),
                   editDistance(str1, str2, m-1, n),
                   editDistance(str1, str2, m-1, n-1)
                   )


In [2]:
str1=input("Enter First String:")
str2=input("Enter Second String:")
print(editDistance(str1, str2, len(str1), len(str2)))

Enter First String:marvel
Enter Second String:marvin
2


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [4]:
data = open('/content/poem.txt', encoding="utf8").read()
corpus = data.lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)

print("Total Words:", total_words)

Total Words: 31


In [5]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)

In [6]:
model = Sequential()
model.add(Embedding(total_words+1, 3, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words+1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 3)              96        
                                                                 
 bidirectional (Bidirection  (None, 8, 300)            184800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 8, 300)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 31)                3131      
                                                                 
 dense_1 (Dense)             (None, 32)                1024      
                                                        

In [10]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
2/2 [==============================] - 0s 47ms/step - loss: 3.8781 - accuracy: 0.2000
Epoch 2/100
2/2 [==============================] - 0s 39ms/step - loss: 3.8615 - accuracy: 0.2000
Epoch 3/100
2/2 [==============================] - 0s 41ms/step - loss: 3.8431 - accuracy: 0.2000
Epoch 4/100
2/2 [==============================] - 0s 43ms/step - loss: 3.8210 - accuracy: 0.2000
Epoch 5/100
2/2 [==============================] - 0s 45ms/step - loss: 3.7993 - accuracy: 0.1636
Epoch 6/100
2/2 [==============================] - 0s 39ms/step - loss: 3.7682 - accuracy: 0.2000
Epoch 7/100
2/2 [==============================] - 0s 45ms/step - loss: 3.7321 - accuracy: 0.1636
Epoch 8/100
2/2 [==============================] - 0s 41ms/step - loss: 3.6896 - accuracy: 0.1636
Epoch 9/100
2/2 [==============================] - 0s 39ms/step - loss: 3.6541 - accuracy: 0.1455
Epoch 10/100
2/2 [==============================] - 0s 43ms/step - loss: 3.6100 - accuracy: 0.1455
Epoch 11/100
2/2 [=

### Poem Generation

In [11]:
seed_text = input("Enter first few words")
next_words = 50
ouptut_text = ""

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1,
      padding='pre')
    predicted = np.argmax(model.predict(token_list,
                                        verbose=0), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    seed_text += " " + output_word

print(seed_text)

Enter first few wordsjoy
joy first great my grief my joys how few few few thee thee few thee thee thee thee thee thee thee thee know thee thee thee thee thee thee thee thee know thee thee thee thee thee thee thee thee know thee thee thee thee thee thee thee thee know thee
